In [1]:
import numpy as np
import numba 
import cython
%load_ext cython
import pandas as pd

numba.__version__, cython.__version__, np.__version__

('0.36.1', '0.27.3', '1.13.3')

In [2]:
X = np.random.random((1000, 3))

In [3]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=np.float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

#%timeit -n10 pairwise_python(X)

In [4]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))

%timeit -n10 pairwise_numpy(X)

64.2 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
pairwise_numba = numba.jit(pairwise_python)

%timeit -n10 pairwise_numba(X)

The slowest run took 8.46 times longer than the fastest. This could mean that an intermediate result is being cached.
14.3 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%cython
import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
def pairwise_cython(double[:, ::1] X):
    cdef int M = X.shape[0]
    cdef int N = X.shape[1]
    cdef double tmp, d
    cdef double[:, ::1] D = np.empty((M, M), dtype=np.float64)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = sqrt(d)
    return D

In [7]:
%timeit -n10 pairwise_cython(X)

7.94 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
def amortize_payments_py(B0, R, term, cpr=0.0):
    smm = 1. - pow(1 - cpr/100., 1/12.)
    r = R/1200.
    S = np.zeros(term)
    P = np.zeros(term)
    I = np.zeros(term)
    B = np.zeros(term)
    Pr = np.zeros(term)
    Bt = B0
    pow_term = pow(1+r, term)
    A = Bt*r*pow_term/(pow_term - 1)
    for i in range(term):
        n = term-i

        I[i] = Bt * r
        Pr[i] = smm*Bt
        S[i] = A-I[i] if Bt>1e-2 else 0.
        P[i] = S[i] + Pr[i]
        Bt = max(Bt - P[i], 0.0)

        B[i] = Bt
    return S,I, Pr,P, B

In [9]:
%%cython
cimport cython 
import numpy as np
from libc.math cimport pow, fmax

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.initializedcheck(False)
def amortize_payments_cy(double B0,double R,int term,double cpr=0.0):
    cdef double smm = 1. - pow(1 - cpr/100., 1/12.)
    cdef double r = R/1200.
    cdef double[:] D = np.empty(term, dtype=np.float64)
    cdef double[:] S = np.empty(term, dtype=np.float64)
    cdef double[:] P = np.empty(term, dtype=np.float64)
    cdef double[:] I = np.empty(term, dtype=np.float64)
    cdef double[:] B = np.empty(term, dtype=np.float64)
    cdef double[:] Pr = np.empty(term, dtype=np.float64)
    cdef double Bt = B0
    cdef double pow_term = pow(1+r, term)
    cdef double A = Bt*r*pow_term/(pow_term - 1.)
    cdef double n = term
    cdef int i=0
    for i in range(term):
        n = term-i
        I[i] = Bt * r
        Pr[i] = smm*Bt
        S[i] = A-I[i] if Bt>1e-2 else 0.
        P[i] = S[i] + Pr[i]
        Bt = fmax(Bt - P[i], 0.0)
        B[i] = Bt
    return S,I,Pr,P,B

In [10]:
amortize_payments_nb = numba.njit(cache=True)(amortize_payments_py)

In [11]:
B0 = 500000.
R = 4.0
term = 360

In [12]:
%timeit -n1000 S,I, Pr,P, B = amortize_payments_py(B0, R, term, cpr=10)

844 µs ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit -n1000 S,I, Pr,P, B = amortize_payments_nb(B0, R, term, cpr=10)

The slowest run took 80.50 times longer than the fastest. This could mean that an intermediate result is being cached.
84.7 µs ± 188 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit -n1000 S,I, Pr,P, B = amortize_payments_cy(B0, R, term, cpr=10)

45.8 µs ± 4.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
